# 버스 사고 예측하기

In [23]:
# !pip install pymysql

In [24]:
# 모듈로드
import json
import os
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sql_setting import *
from sql_connect import Connect_DB

### SQL연결

In [25]:
# 객체 생성
sql_con=Connect_DB(use_DB=False)

In [26]:
# sql에 있는 전체 테이블 이름 출력
sql_con.get_table_names()

['action_dect_df']

In [27]:
# db와 연결한 테이블 불러옴
con_accident_df=sql_con.load_DB('action_dect_df')

In [28]:
con_accident_df

,id,bbox,occluded,num_keypoints,keypoints,action,age,disabled,mask,foldname,json_name,action_int,get_int
0,0,"[681, 436, 1470, 1075]",1,16,"[821, 1075, 0, 740, 986, 1, 979, 1014, 2, 1292...",sit down,adult,0,0,A,[emergency]accident07_001A_0.json,3,2
1,1,"[891, 147, 1007, 554]",1,16,"[945, 515, 2, 936, 433, 2, 933, 344, 2, 973, 3...",stand,adult,0,0,A,[emergency]accident07_001A_0.json,0,2
2,1,"[891, 147, 1007, 554]",1,16,"[945, 515, 2, 936, 433, 2, 933, 344, 2, 973, 3...",stand,adult,0,0,A,[emergency]accident07_001A_10.json,0,2
3,0,"[681, 420, 1470, 1075]",1,16,"[821, 1075, 0, 740, 983, 1, 979, 1012, 2, 1292...",sit down,adult,0,0,A,[emergency]accident07_001A_10.json,3,2
4,1,"[711, 162, 1026, 1034]",1,16,"[814, 1024, 1, 842, 844, 1, 831, 665, 2, 921, ...",walk,adult,0,0,A,[emergency]accident07_001A_100.json,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43901,1,"[611, 747, 1188, 1077]",1,16,"[997, 1077, 0, 997, 1010, 0, 1016, 1042, 0, 88...",walk,adult,0,0,H,[emergency]violence06_036H_550.json,1,2
43902,1,"[1189, 319, 1581, 720]",1,16,"[1458, 636, 1, 1483, 581, 1, 1494, 658, 1, 141...",sit down,adult,0,0,H,[emergency]violence06_036H_60.json,3,2
43903,1,"[1177, 288, 1569, 716]",1,16,"[1446, 632, 1, 1471, 577, 1, 1482, 654, 1, 139...",sit down,adult,0,0,H,[emergency]violence06_036H_70.json,3,2
43904,1,"[1163, 250, 1488, 716]",1,16,"[1373, 670, 1, 1372, 585, 1, 1418, 523, 2, 133...",stand,adult,0,0,H,[emergency]violence06_036H_80.json,0,2


- bbox와 keypoints의 타입은 문자열

In [29]:
# 열 타입변경
keypoins_arr=[]
bbox_arr=[]


for i in range(len(con_accident_df)):
    bbox_arr.append(np.array(list(map(int,con_accident_df.bbox[i].strip('[').strip(']').split(',')))))
    keypoins_arr.append(np.array(list(map(int,con_accident_df.keypoints[i].strip('[').strip(']').split(',')))))
    
con_accident_df.bbox=bbox_arr
con_accident_df.keypoints=keypoins_arr

In [30]:
# keypoints배열 구조 변환
data=[]

for i in range(len(con_accident_df)):
    data.append(con_accident_df.keypoints[i])

data=np.array(data).reshape(-1,16,3)
data

array([[[ 821, 1075,    0],
        [ 740,  986,    1],
        [ 979, 1014,    2],
        ...,
        [1324,  951,    2],
        [1372,  617,    0],
        [1372,  746,    0]],

       [[ 945,  515,    2],
        [ 936,  433,    2],
        [ 933,  344,    2],
        ...,
        [ 989,  232,    1],
        [ 997,  303,    1],
        [ 980,  346,    1]],

       [[ 945,  515,    2],
        [ 936,  433,    2],
        [ 933,  344,    2],
        ...,
        [ 989,  232,    1],
        [ 997,  303,    1],
        [ 980,  346,    1]],

       ...,

       [[1446,  632,    1],
        [1471,  577,    1],
        [1482,  654,    1],
        ...,
        [1333,  424,    1],
        [1268,  454,    1],
        [1298,  368,    1]],

       [[1373,  670,    1],
        [1372,  585,    1],
        [1418,  523,    2],
        ...,
        [1264,  416,    2],
        [1259,  496,    2],
        [1314,  452,    1]],

       [[1331,  670,    1],
        [1337,  561,    1],
        [1381,  

========필요없음========

### 데프만들기 연습

In [31]:
# json파일을 데프로 만들기
# 하나의 폴더안의 하나의 시나리오에 대해서만 가능 함수
labelfold='C:/TeamProject_T3Q/04_code/01_preprocessing/gy/[라벨]Emergency_val_accident/'

def json_df_make(fold,si_fold):
    accident_df=pd.DataFrame()
    folder_li=[]
    path=labelfold+fold+si_fold

    for file in os.listdir(path):
        a_dic=json.load(open(path+file))
        a_dic_ann=a_dic['annotations']
        if len(a_dic_ann)!=0:
            for _ in range(len(a_dic_ann)):
                folder_li.append(fold.split('/')[0])
            accident_df=pd.concat([accident_df,pd.DataFrame(a_dic_ann)],ignore_index=True)
    accident_df['foldname']=folder_li
    
    return accident_df

In [32]:
a=json_df_make('G/','[emergency]accident11_087G/')
a

,id,bbox,occluded,num_keypoints,keypoints,action,age,disabled,mask,get_on,get_off,foldname
0,0,"[1061, 330, 1374, 849]",False,16,"[1092, 771, 2, 1101, 648, 2, 1132, 515, 2, 121...",sit down,adult,False,False,False,False,G
1,0,"[1029, 296, 1365, 842]",False,16,"[1092, 765, 2, 1091, 638, 2, 1149, 505, 2, 123...",sit down,adult,False,False,False,False,G
2,0,"[1030, 272, 1311, 858]",False,16,"[1095, 795, 2, 1092, 639, 2, 1137, 495, 2, 122...",sit down,adult,False,False,False,False,G
3,0,"[1019, 275, 1287, 869]",False,16,"[1093, 804, 2, 1083, 657, 2, 1133, 532, 2, 119...",sit down,adult,False,False,False,False,G
4,0,"[1007, 288, 1282, 878]",False,16,"[1092, 809, 2, 1075, 659, 2, 1120, 524, 2, 119...",sit down,adult,False,False,False,False,G
...,...,...,...,...,...,...,...,...,...,...,...,...
88,0,"[978, 372, 1402, 868]",False,16,"[1102, 792, 2, 1092, 743, 2, 1009, 564, 2, 116...",stand,adult,False,False,False,False,G
89,0,"[967, 373, 1391, 860]",False,16,"[1098, 777, 2, 1095, 730, 2, 1014, 555, 2, 110...",stand,adult,False,False,False,False,G
90,0,"[967, 373, 1391, 860]",False,16,"[1098, 777, 2, 1095, 730, 2, 1014, 555, 2, 110...",stand,adult,False,False,False,False,G
91,0,"[1024, 358, 1380, 858]",False,16,"[1101, 773, 2, 1099, 688, 2, 1085, 538, 2, 118...",stand,adult,False,False,False,False,G


In [38]:
# json파일에 2명의 사람에 대한 데이터가 모두 행으로 변환되는지 확인용
a_dic=json.load(open('[라벨]Emergency_val_accident/E/[emergency]accident12_055E/[emergency]accident12_055E_790.json'))
pd.DataFrame(a_dic['annotations'])

,id,bbox,occluded,num_keypoints,keypoints,action,age,disabled,mask,get_on,get_off
0,0,"[418, 673, 933, 1079]",True,16,"[575, 1079, 0, 575, 982, 0, 864, 978, 2, 805, ...",walk,adult,False,False,False,False
1,1,"[253, 826, 825, 1079]",True,16,"[517, 1027, 0, 517, 963, 0, 517, 899, 0, 134, ...",walk,adult,False,False,False,False


In [39]:
# json파일을 데프로 만들기 함수
# 알파벳은 각각의 카메라를 의미
action_str=['walk','stand','sit down','grab handle','run','kick','punch','lay down']
get_str=['get_off','get_on','nothing']

def json_df_make(fold):
    accident_df=pd.DataFrame()
    folder_li=[]
    json_name_li=[]
    action_int=[]
    get_int=[]

    for f in fold:
        path=labelfold+f
        for si_fo in os.listdir(path):
            files=path+si_fo+'/'
            for file in os.listdir(files):
                accident_dic=json.load(open(files+file))
                accident_dic_ann=accident_dic['annotations']
                if len(accident_dic_ann)!=0:
                    for i in range(len(accident_dic_ann)):
                        # 폴더명(카메라)
                        folder_li.append(f.split('/')[0])

                        # 파일명
                        json_name_li.append(file)

                        # 액션
                        if accident_dic_ann[i]['action']=='walk':
                            action_int.append(0)
                        elif accident_dic_ann[i]['action']=='stand':
                            action_int.append(1)
                        elif accident_dic_ann[i]['action']=='sit down':
                            action_int.append(2)
                        elif accident_dic_ann[i]['action']=='grab handle':
                            action_int.append(3)
                        elif accident_dic_ann[i]['action']=='run':
                            action_int.append(4)
                        elif accident_dic_ann[i]['action']=='kick':
                            action_int.append(5)
                        elif accident_dic_ann[i]['action']=='punch':
                            action_int.append(6)
                        else:
                            action_int.append(7)
                        
                        # 승하차
                        if accident_dic_ann[i]['get_on']==False & accident_dic_ann[i]['get_on']==False:
                            get_int.append(2)
                        elif accident_dic_ann[i]['get_on']==True:
                            get_int.append(1)
                        elif accident_dic_ann[i]['get_off']==True:
                            get_int.append(2)
                accident_df=pd.concat([accident_df,pd.DataFrame(accident_dic_ann)],ignore_index=True)

    accident_df['foldname']=folder_li # 카메라의 알파벳을 새로운 열로 추가
    accident_df['json_name']=json_name_li
    accident_df['action_int']=action_int
    accident_df['get_int']=get_int

    # boolean값 인수형으로 바꾸기
    accident_df.occluded=[1 if b==True else 0 for b in accident_df.occluded]
    accident_df.disabled=[1 if b==True else 0 for b in accident_df.disabled]
    accident_df['mask']=[1 if b==True else 0 for b in accident_df['mask']]

    # 필요없는 열 삭제
    accident_df=accident_df.drop(['get_on','get_off'],axis=1)

    return accident_df

In [41]:
# json폴더안에 있는 모든 json파일을 데프로 만들기
# labelfold='[라벨]Emergency_train_accident/'       # 별도로 폴더 있어야 함. 깃에는 없음요
accident_df=json_df_make(['A/','B/','C/','D/','E/','F/','G/','H/'])

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '[라벨]Emergency_train_accident/A/'

In [ ]:
accident_df

In [ ]:
# csv파일 만들기
accident_df.to_csv('accident.csv',index=False,encoding='cp949')

In [ ]:
# x와 y 나누기
x=np.array(list(accident_df['keypoints'])).reshape(-1,16,3)
y=np.array(list(accident_df['action_int']))

========필요없음========

In [ ]:
# 모듈 로딩
import tensorflow as tf
from keras import Sequential
from keras.layers import Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, BatchNormalization, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [ ]:
# 훈련/테스트 데이트 나누기
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=42,stratify=y)

In [ ]:
# 2D 
model = Sequential()
model.add(Conv2D(32, kernel_size=3, activation='relu', kernel_initializer='he_uniform', input_shape=(16,3,1)))
model.add(MaxPooling2D(pool_size=2,padding='same'))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.5))
# model.add(Conv2D(64, kernel_size=3, activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPooling2D(pool_size=2,padding='same'))
# model.add(BatchNormalization(center=True, scale=True))
# model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(8, activation='softmax'))

# callbacks
es=EarlyStopping(monitor='val_accuracy',patience=10)
mc=ModelCheckpoint('model.h5',monitor='val_acc',save_best_only=True)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])
model.summary()

# Fit data to model
history = model.fit(x_train, y_train,
            batch_size=128,
            epochs=1000,
            verbose=1,
            validation_split=0.2,
            callbacks=[es,mc])

In [ ]:
# train과 val의 정확도 비교 그래프
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.show()

In [ ]:
history.model.predict(np.array([
                1030,
                756,
                2,
                1048,
                612,
                2,
                1071,
                462,
                2,
                1160,
                471,
                2,
                1151,
                624,
                2,
                1134,
                643,
                2,
                1116,
                465,
                2,
                1113,
                279,
                2,
                1138,
                258,
                2,
                1132,
                220,
                2,
                1083,
                206,
                2,
                1021,
                318,
                2,
                1077,
                250,
                1,
                1211,
                281,
                1,
                1180,
                371,
                2,
                1192,
                268,
                2
            ]).reshape(-1,16,3))

In [ ]:
# test데이터 만들기
labelfold='../data/label/'
test_df=json_df_make(['A/','C/','E/','G/'])
test_df

In [ ]:
a=[]

for i in range(len(test_df)):
    a.append(np.array(test_df.keypoints[i]))

a_a=np.array(a)
a_a.reshape(-1,16,3)

In [ ]:
test_df.loc[test_df.action=='lay down',]

In [ ]:
# YOLO로드 => yolo 포맷 및 클래스명 불러오기
net=cv2.dnn.readNet('yolov3.weights','yolov3.cfg') # 네트워크 불러오기
## 1. dnn란 deep neural network로 모듈

# GPU 사용
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# YOLO에 저장된 클래스 종류
classes=[]
with open('coco.names','r') as f: # coco.names의 파일 안에는 다양한 종류의 클래스가 있음
    classes=[line.strip() for line in f.readlines()]

# YOLO 클래스 확인
print(f'파일에 저장되어 있는 클래스 종류\n {classes}\n 종류의 개수={len(classes)}')

# YOLO의 layer확인
layer_names=net.getLayerNames() # 네트워크의 모든 레이어 이름을 가지고 옴 => conv_0, conv_26 등등
output_layers=[layer_names[i-1] for i in net.getUnconnectedOutLayers()]
## 1. net.getUnconnectedOutLayers() : 출력레이어 가져오기
## 2. i-1부분은 출처에는 i[0]-1라고 되어 있는데 오류가 나오는데 왜 [0]을 해줘야 할까??
## 3. YOLO에눈 3개의 출력 레이어가 있음 => 82/94/106

# 랜덤으로 색 지정하기 => RGB
colors=np.random.uniform(0,255,size=(len(classes),3))
## 1. 80행 3열로 된 0~255인 균일분포의 값을 랜덤 추출

In [ ]:
def img_bou(path):
    # 이미지 불러오기
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    # img=cv2.resize(img,(850,850),fx=0.4,fy=0.4)
    height,width,channels=img.shape

    # 객체 탐지
    blob = cv2.dnn.blobFromImage(img, 0.00392, (608, 608), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # 바운딩 박스 죄표 찾기
    class_ids = []
    confidences = [] 
    boxes = [] 

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if class_id==0 and confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    # 노이즈 제거
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # 화면에 표시
    font = cv2.FONT_HERSHEY_PLAIN
    boxes_index=[] 

    for i in range(len(boxes)):
        if i in indexes:
            boxes_index.append(boxes[i]) 
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2) 
            cv2.putText(img, label, (x, y + 30), font, 3, color, 3)

    print(boxes_index)
    plt.imshow(img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [ ]:
img_bou('busuksa4.jpg')

In [ ]:
# test데이터로 예측하기
def test_pre(json_name):
    dect_df=test_df[test_df.json_name==json_name]
    path='C:/Users/USER/Desktop/com_project/data/rawdata/'
    plt.figure(figsize=(20,5))
    s=1
    for i in dect_df.index:
        acc=json_name.split('_')[0]
        sina=json_name.split('_')[1]
        fold=acc+'_'+sina
        plt.subplot(1,len(dect_df),s)
        s+=1
        img_bou(path+sina[-1]+'/'+fold+'/'+json_name.split('.')[0]+'.jpg')
        val=np.array(dect_df.loc[i,'keypoints']).reshape(-1,16,3)
        pre=action_str[np.argmax(history.model.predict(val))]
        print(f"실제값은{dect_df.loc[i,'action']} 예측값은{pre}")

In [ ]:
dect_df=test_df[test_df.json_name=='[emergency]accident12_056E_600.json']
dect_df.index

In [ ]:
img_bou('C:/Users/USER/Desktop/com_project/data/rawdata/G/[emergency]accident12_056G/[emergency]accident12_056G_230.jpg')

In [ ]:
test_pre('[emergency]accident12_056G_180.json')

In [ ]:
test_pre('[emergency]accident12_056G_250.json')

In [ ]:
test_pre('[emergency]accident12_056E_610.json')